In [1]:
import re
from os import listdir
from os.path import isfile, join
import csv
import os
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
train_dir = '/home/hoang/Downloads/train/'
files = [f for f in listdir(train_dir) if isfile(join(train_dir, f))]

class CSVProcessor:
    def __init__(self, fdir, fname):
        self.fdir = fdir
        self.fname = fname
        
    def process(self):
        line = 0
        df = pd.read_csv(filepath_or_buffer=os.path.join(self.fdir, self.fname))
        return df
    
proc = CSVProcessor('data/', 'trainLabels.csv')
df = proc.process()

train, test = train_test_split(df, test_size=0.8, random_state=42) # train on 40% given data

In [2]:
train_hashes = [x for x in train['Id']]
train_classes = [x for x in train['Class']]

In [7]:
class GetDf:
    def __init__(self, data):
        self.hashes = [x for x in data['Id']]
        self.classes = [x for x in data['Class']]
        
    def get_df(self):
        df = pd.DataFrame(columns=['hashes', 'asm', 'bytes', 'class'])
        df['hashes'] = self.hashes
        df['class'] = self.classes
        df['asm'] = [os.stat(join(train_dir, f + '.asm')).st_size for f in self.hashes]
        df['bytes'] = [os.stat(join(train_dir, f + '.bytes')).st_size for f in self.hashes]
        
        scaler = StandardScaler()
        scaler.fit(df[['asm', 'bytes']])
        df[['asm', 'bytes']] = scaler.transform(df[['asm', 'bytes']])
        
        return df

In [8]:
from sklearn.ensemble import RandomForestClassifier

rdc = RandomForestClassifier(n_estimators=400, max_depth=int(np.sqrt(2173)))

In [9]:
train_df = GetDf(train).get_df()

In [10]:
train_df.shape

(2173, 4)

In [11]:
rdc.fit(train_df[['asm', 'bytes']], train_df[['class']])

/home/hoang/Documents/school/4400/env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=46, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
from sklearn.metrics import accuracy_score

test_df = GetDf(test).get_df()
print(test_df)
accuracy_score(rdc.predict(test_df[['asm', 'bytes']]), test_df['class'])

                    hashes       asm     bytes  class
0     E4lBhCXvziDf2rU3GM7t -0.553289  1.011585      3
1     k0DVI4svgBRA8pjlf9M1 -0.375412 -0.945791      1
2     c64G93TVZM258AUDSCka -0.356998 -1.005105      2
3     cpdsREmet3FyD1oYZVIn -0.557265  1.011585      3
4     283STBsQaI7Pw9qf0tjy -0.524947 -1.139551      2
5     CW6cHidJb35VTMQ0jms7 -0.557271  1.011585      3
6     BAGfw1SeoPIqyhTErbit -0.432004 -1.100008      4
7     6kgpyCID8l4Pw7BYHTjM -0.389260 -1.060465      8
8     gGbvOKAXCB5047PEf6aS -0.467810 -1.076282      1
9     F3Zj217CLRxgi0NyHMY4 -0.497485 -1.076282      1
10    FSqi0ZaPveb5jMxtBcfs -0.230800 -1.032785      9
11    Agr9Oh1eLIP0QCpRNkUt  0.879685  1.011585      3
12    GkA3vhfoHbwdeKPBl4ZR -0.239741 -1.032785      9
13    kGITL4OJxYMWEQ1bKBiP  0.326556 -0.408007      9
14    fgD5k647jbdSEWBZCQnh -0.557277  1.015539      3
15    ic1gqvXzQFDjZ5YxkmSt -0.556988  1.766855      3
16    blhHt3mGR6pFC0Or41SL -0.557053  1.766855      3
17    BC9hzqbTa0OIsePYiND6  

0.5709028177113283

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7).fit(train_df[['asm', 'bytes']], train_df[['class']])

In [ ]:
knn.score(test_df[['asm', 'bytes']], test_df[['class']])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=200)

In [ ]:
ada.fit(train_df[['asm', 'bytes']], train_df['class'])

In [ ]:
ada.score(test_df[['asm', 'bytes']], test_df['class'])

In [ ]:
dtc = DecisionTreeClassifier(max_depth=10)
dtc.fit(train_df[['asm', 'bytes']], train_df['class'])

In [ ]:
dtc.score(test_df[['asm', 'bytes']], test_df['class'])